# Feature Engineering after Data Exploration

After exploring some of the data, there remained several metrics that we were unable to explore due to formatting such as:
- Text-based data with unique entries
- Time-based data
- Unstructed image-based data

Therefore, it may now be worthwhile to generate new features that summarize the above metrics easier.

## Setup

In [2]:
# Setup API
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import config as cfg
import os

sa_credentials = 'gpc/youtube-scraper-404402-a6dc21ea107a.json' # download from service_account page
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = sa_credentials

credentials = service_account.Credentials.from_service_account_file(
    sa_credentials, scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Read tables into python
query_string = """
SELECT 
    *
FROM youtube-scraper-404402.yt_data.video_metrics_cleaned
"""
video_metrics_df = bq_client.query(query_string).to_dataframe()

query_string = """
SELECT 
    *
FROM youtube-scraper-404402.yt_data.video_comments_cleaned
"""
video_comments_df = bq_client.query(query_string).to_dataframe()

In [3]:
video_metrics_df.head()

,int64_field_0,channel_name,video_id,title,published_at,description,thumbnail_url,tags,category_id,duration,view_count,like_count,fav_count,comment_count
0,3,Why I Salty,K3s5zrG4t1c,"Got a Job for you, 621: Mission 11 | Armored C...",2023-11-12 05:00:08+00:00,#armoredcore6 #bobthebuilder \n\nAI can only d...,https://i.ytimg.com/vi/K3s5zrG4t1c/default.jpg,"['armored core', 'Gundam', 'Anime', 'ac6', 'Ga...",20,PT1M23S,4829,549,0,50
1,4,Why I Salty,HypJk4azqAE,Pepperidge Farm Remember | Armored Core VI,2023-11-11 03:58:05+00:00,#armoredcore6 #familyguy #asmongold \n\nJust t...,https://i.ytimg.com/vi/HypJk4azqAE/default.jpg,"['armored core', 'Gundam', 'Anime', 'ac6', 'Ga...",20,PT22S,2524,239,0,24
2,14,Why I Salty,tsmFLhITvGY,"Got a Job for you, 621: Mission 10 | Armored C...",2023-11-04 04:00:25+00:00,#armoredcore6 @SpongeBobOfficial \n\nDecided t...,https://i.ytimg.com/vi/tsmFLhITvGY/default.jpg,"['armored core', 'Gundam', 'Anime', 'ac6', 'Ga...",20,PT2M3S,7127,613,0,37
3,17,Why I Salty,qls8YCcN8T4,WE DID IT! Asmongold Reacted to EP3,2023-11-01 21:05:17+00:00,#armoredcore6 #asmongold #mcdonalds \n\nI neve...,https://i.ytimg.com/vi/qls8YCcN8T4/default.jpg,"['armored core', 'Gundam', 'Anime', 'ac6', 'Ga...",20,PT8M37S,6187,912,0,61
4,19,Why I Salty,Hv_dkdAp8SQ,Not Part of Your World | Armored Core VI,2023-11-01 05:36:37+00:00,"#armoredcore6 \n\nWhen i made the meme ""Booty ...",https://i.ytimg.com/vi/Hv_dkdAp8SQ/default.jpg,"['armored core', 'Gundam', 'Anime', 'ac6', 'Ga...",20,PT1M29S,9400,939,0,91


In [4]:
video_comments_df.head()

,int64_field_0,video_id,video_comment,video_comment_user,video_comment_time,video_comment_parent_id,video_comment_child_id
0,0,HqNMNXa7REY,Good luck trying out the Charge Blade. Transfo...,Gobo KoboObo,2023-11-15 05:10:21+00:00,UgzWhoTNYW-C1nD1Lm94AaABAg,0
1,1,P1a3DEpvFeE,"Nice, good luck on the rest of your journey, s...",Diego Cantu,2023-11-14 03:48:22+00:00,Ugx5R_BBSeGsBuC3AN54AaABAg,0
2,2,P1a3DEpvFeE,Stream died?,Gobo KoboObo,2023-11-14 01:46:07+00:00,Ugx4-6nI0vtlaGv735d4AaABAg,0
3,3,P1a3DEpvFeE,lel so it wasn't just me then XD,Kuruwa Yoshiko,2023-11-14 01:47:07+00:00,Ugx4-6nI0vtlaGv735d4AaABAg,Ugx4-6nI0vtlaGv735d4AaABAg.9x4SL68zJEO9x4SSUbjmUM
4,4,P1a3DEpvFeE,@Kuruwa Yoshiko Thought it was my internet.,Gobo KoboObo,2023-11-14 01:50:20+00:00,Ugx4-6nI0vtlaGv735d4AaABAg,Ugx4-6nI0vtlaGv735d4AaABAg.9x4SL68zJEO9x4Sp7-cGGN


## Text-based Features

For both tables, let's look at how we can extract meaninful information from:
- Video Title
- Video Description
- Video Tags
- Video Comments

### Video Title

Let's look at the first 10 video titles:

In [5]:
for title in video_metrics_df['title'].head(10):
    print(title)

Got a Job for you, 621: Mission 11 | Armored Core VI
Pepperidge Farm Remember | Armored Core VI
Got a Job for you, 621: Mission 10 | Armored Core VI
WE DID IT!  Asmongold Reacted to EP3
Not Part of Your World | Armored Core VI
So Close Yet So Far | Asmongold No React
What does the Raven Say? | Armored Core VI
Got a Job for you, 621: Mission 9 | Armored Core VI
Got a Job for you, 621: Mission 8 | Armored Core VI
Got a Job for you, 621: Mission 7 | Armored Core VI


It seems like videos might contain the subject matter in the title (such as which video game is being played), let's explore that in more detail.

In [6]:
video_titles = video_metrics_df['title']
subject_matter = []
for title in video_titles:
    if ' | ' in title:
        subject_matter.append(title.split(' | ')[1])
    else:
        subject_matter.append('None')
video_metrics_df['subject_matter'] = subject_matter
video_metrics_df.value_counts('subject_matter')


subject_matter
None                              390
Armored Core VI                    29
Eternal Card Game                  23
Titan Core                         22
Mechabellum                        10
Monster Hunter World [STREAM]       4
Mechabellum [STREAM]                3
Titian Core                         3
Armored Core 2 [STREAM]             3
Armored Core VI [STREAM]            2
Armored Core 3 [STREAM]             2
Eternal Card Game [STREAM]          2
Titan Fall 2 [STREAM]               1
Titan Core (My Game)                1
Spell Slingers                      1
Monster Hunter World                1
Armored Core 2 [FINALE STREAM]      1
MGS3 Snake Eater                    1
Mecabellum [STREAM]                 1
MSG BOP 2 [STREAM]                  1
League of Legends                   1
Asmongold Reacts                    1
Asmongold No React                  1
Armored Core Vi[STREAM]             1
Armored Core 3[STREAM]              1
Armored Core 2 [Stream]            

We may need to reformat some of these for consistency, but let's also make sure that we're not missing anything for titles labeled as 'None'

In [7]:
video_metrics_df[video_metrics_df['subject_matter'] == 'None']['title']

3                   WE DID IT!  Asmongold Reacted to EP3
15                                          Ugh, Fine...
17                                        Raven Please..
18     Got a Job For you 621 (6) #armoredcore6 #armor...
19           How I clone voices for AC6 (Not a Tutorial)
                             ...                        
496    Money Brew - Clockroach Prophecy [Eternal Card...
497       Money Brew - Jurassic Park [Eternal Card Game]
499                                           Thanks You
500    Got a Job For you 621 (4) #armoredcore6 #armor...
501    Salty Brew - Subversion of Dichro [Eternal Car...
Name: title, Length: 390, dtype: object

It looks like that other videos contain the subject matter as hashtags or enclosed in square brackets. Since there are multiple hashtags in a title, we'll need to ingnores those are try to look in the tags section insted, however let's try to pull out those enclosed in square brackets.

In [8]:
import re

video_titles = video_metrics_df['title']
subject_matter = []
for title in video_titles:
    if ' | ' in title:
        subject_matter.append(title.split(' | ')[1])
    elif '[' in title and ']' in title:
        subject_matter.append(re.sub('.*.\[', '', re.sub('\].*.|\]', '', title)))
    else:
        subject_matter.append('None')
        
video_metrics_df['subject_matter'] = subject_matter
video_metrics_df.value_counts('subject_matter')

subject_matter
Eternal Card Game                 282
None                               77
Armored Core VI                    29
Titan Core                         22
Mythgard                           16
Mechabellum                        10
Legend of Runeterra                 9
STREAM                              7
Monster Hunter World [STREAM]       4
Mechabellum [STREAM]                3
YGO Master Duel                     3
Titian Core                         3
MTG Arena                           3
MTG ARENA                           3
Armored Core 2 [STREAM]             3
Throne                              3
Gods Unchained                      2
Storybook Brawl                     2
Eternal Card Game [STREAM]          2
Armored Core VI [STREAM]            2
Armored Core 3 [STREAM]             2
Spell Slingers                      1
Mythgard Card Game                  1
Monster Hunter World                1
Titan Core (My Game)                1
Titan Fall 2 [STREAM]              

Now we've significantly reduced the number of videos with no subject matter as it looks like videos based on the Eternal Card Game were enclosed in square brackets. Again, let's check if there is anything else that needs to be extracted from titles with no subject matter.

In [9]:
video_metrics_df[video_metrics_df['subject_matter'] == 'None']['title'].head(35)

3                   WE DID IT!  Asmongold Reacted to EP3
15                                          Ugh, Fine...
17                                        Raven Please..
18     Got a Job For you 621 (6) #armoredcore6 #armor...
19           How I clone voices for AC6 (Not a Tutorial)
20     Got a Job For you 621 (5) #armoredcore6 #armor...
30          Time to Choose Raven (A.I Voice Test Ver 2).
31                        MegasXLR + Armored Core 6 (PC)
32          Raven Don't Listen to Ayre (A.I Voice Test).
33                Gods Unchained - Selling my first card
40                                  Mechabellum 7/5/2023
42                                 Mechbellum First Try!
51           Project Champion - Light the Fuse (eternal)
52     Project Champion - Updated my Deck Editor & Do...
53                                 I Have Pateron Now!!!
54       HearthSalt Battle Grounds - Dancing with Demons
55     HearthSalt Battlegrounds - I have no idea what...
57                             

In [10]:
video_metrics_df[video_metrics_df['subject_matter'] == 'None']['title'].tail(35)

187                       Gods Unchained - Magic Starter
193                      Why I Valorant? - Beta Brothers
202                               Eternal Stream 7/5/203
207                                        True Rockstar
208                                    DRAW NICO CADO!!!
209                                    My Test Broadcast
219                                   100 Subscribers!!!
220                  I believe in the heart of the cards
223                          Project Champion my own DCG
266                               Armor Core VI Stream 1
267                            I'm a filthily casual now
279                       Gods Unchained - Light Starter
280        Project Champion - Geminon's Choice (eternal)
284    I became partnered in Mythgard + 100 packs ope...
287                                Titan Core Needs You!
291                   Gods Unchained - First Impressions
292           Eternal - Seneschal of Light combo Attempt
327                      Sallty

It looks like some videos also have the subject matter prior to the title, seperated by the '-' character. Let's try to grab those and then take a look at the final list.

In [11]:
video_titles = video_metrics_df['title']
subject_matter = []
for title in video_titles:
    if ' | ' in title:
        subject_matter.append(title.split(' | ')[1])
    elif '[' in title and ']' in title:
        subject_matter.append(re.sub('.*.\[', '', re.sub('\].*.|\]', '', title)))
    elif ' - ' in title:
        subject_matter.append(re.sub(' - .*.', '', title))
    else:
        subject_matter.append('None')
        
video_metrics_df['subject_matter'] = subject_matter
video_metrics_df.value_counts('subject_matter')

subject_matter
Eternal Card Game                 282
None                               46
Armored Core VI                    29
Titan Core                         22
Mythgard                           17
Mechabellum                        10
Gods Unchained                     10
Legend of Runeterra                 9
STREAM                              7
Project Champion                    6
Project Champions                   5
Monster Hunter World [STREAM]       4
Mechabellum [STREAM]                3
Armored Core 2 [STREAM]             3
MTG ARENA                           3
MTG Arena                           3
Throne                              3
YGO Master Duel                     3
Titian Core                         3
Hearthstone                         2
Armored Core 3 [STREAM]             2
Armored Core VI [STREAM]            2
Why I Valorant?                     2
Eternal                             2
Eternal Card Game [STREAM]          2
Storybook Brawl                    

Let's look at titles with 'None' for subject title one more time and see if we can simply pull keywords.

In [12]:
video_metrics_df[video_metrics_df['subject_matter'] == 'None']['title'].to_list()

['WE DID IT!  Asmongold Reacted to EP3',
 'Ugh, Fine...',
 'Raven Please..',
 'Got a Job For you 621 (6) #armoredcore6 #armoredcore #gaming',
 'How I clone voices for AC6 (Not a Tutorial)',
 'Got a Job For you 621 (5) #armoredcore6 #armoredcore #gaming',
 'Time to Choose Raven (A.I Voice Test Ver 2).',
 'MegasXLR + Armored Core 6 (PC)',
 "Raven Don't Listen to Ayre (A.I Voice Test).",
 'Mechabellum 7/5/2023',
 'Mechbellum First Try!',
 'I Have Pateron Now!!!',
 'Eternal Stream 7/4/2023',
 'Mechabellum Stream 07/03/23',
 'Hey Check out my brother gaming channel!',
 "This is why we don't rope people. (No audio)",
 'Milos Monstars',
 'Eternal Tournament recording test (Salty Games)',
 'Eternal Stream 7/4/2023 part 2',
 'When they nerf they will nerf it hard.',
 'Eternal Stream 06/18/23',
 'Vanguard Bandits Playthrough #2',
 'Eternal Stream 06/19/23',
 'Lynax Combo',
 'The nerfs of EHG',
 'Eternal Stream 7/5/203',
 'True Rockstar',
 'DRAW NICO CADO!!!',
 'My Test Broadcast',
 '100 Subscrib

Based on the above and the subject matter list, we can pull the following:
- Armored Core
- Mechabellum
- Eternal
- Project Champion
- Mythgard
- Titan Core
- Monster Hunter

There are some typos for some of these titles but we'll leave them alone as that may potentially influence the YouTube algorithm.

In [13]:
video_titles = video_metrics_df['title']
subject_matter = []
for title in video_titles:
    if ' | ' in title:
        subject_matter.append(title.split(' | ')[1])
    elif '[' in title and ']' in title:
        subject_matter.append(re.sub('.*.\[', '', re.sub('\].*.|\]', '', title)))
    elif ' - ' in title:
        subject_matter.append(re.sub(' - .*.', '', title))
    elif 'armored' in title.lower():
        subject_matter.append('Armored Core')
    elif 'mechabellum' in title.lower():
        subject_matter.append('Mechabellum')
    elif 'eternal' in title.lower():
        subject_matter.append('Eternal Card Game')
    elif 'project champion' in title.lower():
        subject_matter.append('Project Champion')
    elif 'mythgard' in title.lower():
        subject_matter.append('Mythgard')
    elif 'titan core' in title.lower():
        subject_matter.append('Titan Core')
    elif 'monster hunter' in title.lower():
        subject_matter.append('Monster Hunter')
    else:
        subject_matter.append('None')
        
video_metrics_df['subject_matter'] = subject_matter
video_metrics_df.value_counts('subject_matter')

subject_matter
Eternal Card Game                 288
Armored Core VI                    29
None                               27
Titan Core                         23
Mythgard                           18
Mechabellum                        12
Gods Unchained                     10
Legend of Runeterra                 9
Armored Core                        7
Project Champion                    7
STREAM                              7
Project Champions                   5
Monster Hunter World [STREAM]       4
Throne                              3
Mechabellum [STREAM]                3
Titian Core                         3
YGO Master Duel                     3
MTG ARENA                           3
MTG Arena                           3
Armored Core 2 [STREAM]             3
Armored Core 3 [STREAM]             2
Storybook Brawl                     2
Eternal                             2
Eternal Card Game [STREAM]          2
Armored Core VI [STREAM]            2
Why I Valorant?                    

One last sanity check for videos with no subject matter in the title:

In [14]:
video_metrics_df[video_metrics_df['subject_matter'] == 'None']['title'].to_list()

['WE DID IT!  Asmongold Reacted to EP3',
 'Ugh, Fine...',
 'Raven Please..',
 'How I clone voices for AC6 (Not a Tutorial)',
 'Time to Choose Raven (A.I Voice Test Ver 2).',
 "Raven Don't Listen to Ayre (A.I Voice Test).",
 'Mechbellum First Try!',
 'I Have Pateron Now!!!',
 'Hey Check out my brother gaming channel!',
 "This is why we don't rope people. (No audio)",
 'Milos Monstars',
 'When they nerf they will nerf it hard.',
 'Vanguard Bandits Playthrough #2',
 'Lynax Combo',
 'The nerfs of EHG',
 'True Rockstar',
 'DRAW NICO CADO!!!',
 'My Test Broadcast',
 '100 Subscribers!!!',
 'I believe in the heart of the cards',
 'Armor Core VI Stream 1',
 "I'm a filthily casual now",
 'Decided to buy Collector on Udyr.',
 'I just can’t wait any longer!',
 'How i felt when i finally cast Lastlight judgement in a game',
 'Vanguard Bandits Playthrough #1',
 'Thanks You']

Great! Let's also clean up the subject matter for topics with 5 or more videos, the rest we can classify as 'Misc.'.

In [15]:
# Clean up variations of each subject matter
video_metrics_df.loc[video_metrics_df['subject_matter'].str.contains('Eternal'), 'subject_matter'] = 'Eternal Card Game'
video_metrics_df.loc[video_metrics_df['subject_matter'].str.contains('Armored Core'), 'subject_matter']= 'Armored Core'
video_metrics_df.loc[video_metrics_df['subject_matter'].str.contains('Titan Core'), 'subject_matter'] = 'Titan Core'
video_metrics_df.loc[video_metrics_df['subject_matter'].str.contains('Mechabellum'), 'subject_matter'] = 'Mechabellum'
video_metrics_df.loc[video_metrics_df['subject_matter'].str.contains('Project Champion'), 'subject_matter']= 'Project Champions'
video_metrics_df.loc[video_metrics_df['subject_matter'].str.contains('Monster Hunter'), 'subject_matter'] = 'Monster Hunter'
video_metrics_df.loc[video_metrics_df['subject_matter'].str.lower().str.contains('mtg arena'), 'subject_matter'] = 'MTG Arena'
video_metrics_df.loc[video_metrics_df['subject_matter'].str.contains('Hearthstone'), 'subject_matter'] = 'Hearthstone'

# Get subject matter in 5 or more videos
subject_matter_count = video_metrics_df['subject_matter'].value_counts()
popular_subjects = subject_matter_count[subject_matter_count >= 5].index.to_list()

video_metrics_df.loc[~video_metrics_df['subject_matter'].isin(popular_subjects), 'subject_matter'] = 'Misc.'

video_metrics_df['subject_matter'].value_counts()

Eternal Card Game      293
Armored Core            47
Misc.                   34
None                    27
Titan Core              24
Mythgard                18
Mechabellum             15
Project Champions       12
Gods Unchained          10
Legend of Runeterra      9
STREAM                   7
Monster Hunter           6
MTG Arena                6
Name: subject_matter, dtype: int64

In [16]:
video_metrics_df.loc[video_metrics_df['subject_matter'] == 'STREAM', 'title']

36                           Trying Out New Rig [STREAM]
97     Still trying to have fun in Expedition  Eterna...
135    Just Making and Adjusting Cards in Titan Core ...
140           Mecha Chess then Hunting Monsters [STREAM]
141          Working on my card game Titan Core [STREAM]
164    Making Fast Food Emblems and then Hunting Mons...
471              Playing AC2 for the first time [STREAM]
Name: title, dtype: object

The STREAM subject matter seems to be a side effect of the square bracket format, let's assign those the appropriate subject matter manually:

In [17]:
video_metrics_df.loc[video_metrics_df['subject_matter'] == 'STREAM', 'subject_matter'] = ['None', 'Eternal Card Game', 'Titan Core', 'Monster Hunter', 'Titan Core', 'Monster Hunter', 'Armored Core']
video_metrics_df['subject_matter'].value_counts()

Eternal Card Game      294
Armored Core            48
Misc.                   34
None                    28
Titan Core              26
Mythgard                18
Mechabellum             15
Project Champions       12
Gods Unchained          10
Legend of Runeterra      9
Monster Hunter           8
MTG Arena                6
Name: subject_matter, dtype: int64

While we're here, let's make a separate feature based on whether the video was streamed or not based on the title.

In [18]:
video_metrics_df['streamed'] = [1 if 'STREAM' in x.upper() else 0 for x in video_metrics_df['title']]
video_metrics_df['streamed'].value_counts()

0    470
1     38
Name: streamed, dtype: int64

### Video Description

Let's take a look at the first 10 video description entries:

In [19]:
video_metrics_df['description'].head(10).to_list()

["#armoredcore6 #bobthebuilder \n\nAI can only do so much. I sure hope when my channel grows so I can hire people to voice act.  Some might get confuse, but bob didn't give Manny the job he stole it from him. Since that sentience can be interpreted as Bob giving a job to Manny as well.\n\n\nArt for the thumbnail was made using A.I. Which you can get the full art on my Patreon. As well actually art that was made form my past videos.\n\nPatreon: https://www.patreon.com/WhyISalty\nTwitch: https://m.twitch.tv/whyisalty/profile\nTwitter: https://twitter.com/WhyISalty1\nDiscord: discord.gg/fUTS7w6",
 '#armoredcore6 #familyguy #asmongold \n\nJust thought of this idea all of a sudden and I just had to make it. It turn out ok. Maybe i might do more of these.\n\nPatreon: https://www.patreon.com/WhyISalty\nTwitch: https://m.twitch.tv/whyisalty/profile\nTwitter: https://twitter.com/WhyISalty1\nDiscord: discord.gg/fUTS7w6',
 '#armoredcore6 @SpongeBobOfficial \n\nDecided to make a proper version of 

Based on this, there's a couple of things we can do:
- Remove hashtags and special characters to clean the text for downstream NLP sentiment analysis.
- Create three features based on the presence of hashtags, timestamps, and socials

In [20]:
video_metrics_df['description_text_only'] = [re.sub('#.* |#.*|\\n|\\\'|([0-9]|[0-9][0-9]):.*.', '', x) for x in video_metrics_df['description']]
video_metrics_df['description_hashtags'] = [1 if '#' in x else 0 for x in video_metrics_df['description']]
video_metrics_df['description_timestamps'] = [1 if re.search('([0-9]|[0-9][0-9]):', x) else 0 for x in video_metrics_df['description']]
video_metrics_df['description_socials'] = [1 if re.search('discord|patreon|twitch|twitter', x.lower()) else 0 for x in video_metrics_df['description']]

Now let's confirm each one:

In [21]:
video_metrics_df['description_text_only'].head().to_list()

['AI can only do so much. I sure hope when my channel grows so I can hire people to voice act.  Some might get confuse, but bob didnt give Manny the job he stole it from him. Since that sentience can be interpreted as Bob giving a job to Manny as well.Art for the thumbnail was made using A.I. Which you can get the full art on my Patreon. As well actually art that was made form my past videos.Patreon: https://www.patreon.com/WhyISaltyTwitch: https://m.twitch.tv/whyisalty/profileTwitter: https://twitter.com/WhyISalty1Discord: discord.gg/fUTS7w6',
 'Just thought of this idea all of a sudden and I just had to make it. It turn out ok. Maybe i might do more of these.Patreon: https://www.patreon.com/WhyISaltyTwitch: https://m.twitch.tv/whyisalty/profileTwitter: https://twitter.com/WhyISalty1Discord: discord.gg/fUTS7w6',
 'Decided to make a proper version of the very first "Got a Job" meme Ive made. Since that one was just a quick test i put together. Also this video a another test to where i 

In [22]:
video_metrics_df[video_metrics_df['description_hashtags'] == 1]['description'].head()

0    #armoredcore6 #bobthebuilder \n\nAI can only d...
1    #armoredcore6 #familyguy #asmongold \n\nJust t...
2    #armoredcore6 @SpongeBobOfficial \n\nDecided t...
3    #armoredcore6 #asmongold #mcdonalds \n\nI neve...
4    #armoredcore6 \n\nWhen i made the meme "Booty ...
Name: description, dtype: object

In [23]:
video_metrics_df[video_metrics_df['description_timestamps'] == 1]['description'].head().to_list()

['A brew i made for my first Twitch Subscriber "InsurmountableOne". Since he subbed me on twitch i have to do two brews for him in total. For the first brew he as me to try to make a deck that work around using "Lada, Svetya\'s Advisor". This was an interesting request and hard one to do with all of the control decks out there. I would not recommend using this for climbing. Also when i was recording the intro my webcam freeze for some reason.\n\nIf you want me to brew a deck for you then join my pateron and the next four people to sign up for either the small/medium size fries tiers on my Patreon will automatic be updated to large size fries tier for the price of the tier they sign up for.\n\n0:00 Intro\n4:11 MaskofComedy [FJ]\n11:52 VirginMary [PS]\n23:06 slashH [JS]\n28:54 Kaasplank [TJP]\n35:35 RagMan [JP]\n42:44 Samwisjus [FJ]\n \n\nDeck list:https://eternalwarcry.com/decks/d/EroPwXYbsC4/money-brew-site-ya-lada\nPateron: https://www.patreon.com/WhyISalty\nDiscord: https://discord.g

In [24]:
video_metrics_df[video_metrics_df['description_socials'] == 1]['description'].head().to_list()

["#armoredcore6 #bobthebuilder \n\nAI can only do so much. I sure hope when my channel grows so I can hire people to voice act.  Some might get confuse, but bob didn't give Manny the job he stole it from him. Since that sentience can be interpreted as Bob giving a job to Manny as well.\n\n\nArt for the thumbnail was made using A.I. Which you can get the full art on my Patreon. As well actually art that was made form my past videos.\n\nPatreon: https://www.patreon.com/WhyISalty\nTwitch: https://m.twitch.tv/whyisalty/profile\nTwitter: https://twitter.com/WhyISalty1\nDiscord: discord.gg/fUTS7w6",
 '#armoredcore6 #familyguy #asmongold \n\nJust thought of this idea all of a sudden and I just had to make it. It turn out ok. Maybe i might do more of these.\n\nPatreon: https://www.patreon.com/WhyISalty\nTwitch: https://m.twitch.tv/whyisalty/profile\nTwitter: https://twitter.com/WhyISalty1\nDiscord: discord.gg/fUTS7w6',
 '#armoredcore6 @SpongeBobOfficial \n\nDecided to make a proper version of 

### Video Comments

Similar to the Title and Description, let's also look at the comments.

In [25]:
video_comments_df['video_comment'].head(20).to_list()

['Good luck trying out the Charge Blade. Transforming weapon are too complicated for my lance brain.',
 'Nice, good luck on the rest of your journey, some of the hunts are really tough later on but you got this 👍',
 'Stream died?',
 "lel so it wasn't just me then XD",
 '@Kuruwa Yoshiko Thought it was my internet.',
 "@Why I Salty Ah, that's unfortunate mate. Was thinking about jumping in.",
 '\u200b@Why I SaltyGood that you found a solution. And apparently my laptop can handle this game in low setting pretty well. I can help you tank if you hit a roadblock 👍',
 "Who's the thumbnail artist? Can't quite work out what is going on with the jaw in the top right",
 "@Why I Salty oh buddy, that's not good, if you want to build a community, you shouldn't stab the creative community in the back like this, even just a screenshot from the video would probably be better",
 '“Can you kill him yes you can” ah that gives me robot chicken vibes',
 'Most sane mission in AC6',
 "Ayre wearing coveralls i

In [26]:
video_comments_df['video_comment'].tail().to_list()

[None, None, None, None, None]

Similar to the video description, we need to remove the special characters. It also seems that during the data cleaning step, that we accidently introduced some NoneType variables - let's remove those rows since they represent videos without comments.

In [27]:
video_comments_df = video_comments_df.dropna()
video_comments_df.loc[:,'video_comment'] = [re.sub('\\n|\\\'', '', x) for x in video_comments_df['video_comment']]

In [28]:
video_comments_df['video_comment'].head(20).to_list()

['Good luck trying out the Charge Blade. Transforming weapon are too complicated for my lance brain.',
 'Nice, good luck on the rest of your journey, some of the hunts are really tough later on but you got this 👍',
 'Stream died?',
 'lel so it wasnt just me then XD',
 '@Kuruwa Yoshiko Thought it was my internet.',
 '@Why I Salty Ah, thats unfortunate mate. Was thinking about jumping in.',
 '\u200b@Why I SaltyGood that you found a solution. And apparently my laptop can handle this game in low setting pretty well. I can help you tank if you hit a roadblock 👍',
 'Whos the thumbnail artist? Cant quite work out what is going on with the jaw in the top right',
 '@Why I Salty oh buddy, thats not good, if you want to build a community, you shouldnt stab the creative community in the back like this, even just a screenshot from the video would probably be better',
 '“Can you kill him yes you can” ah that gives me robot chicken vibes',
 'Most sane mission in AC6',
 'Ayre wearing coveralls is some

In [29]:
video_comments_df['video_comment'].tail().to_list()

['I put that legendary that kills all attachments in one deck and its actually not a bad thing it kils your own too, ive had it kill 8 attachments vs a curse deck and including my own and it got charge, warcry, double damage, berserk etc etc you get the idea and won me the game right away - it did 20 damage - 5 doubled (10) + berserk to attack again',
 'Nice, and I accidently played my expedition version in throne too and it also did surprisingly well. As I keep running into endurance I play inflict conscience over permafrost.  A small point of play that Ive noticed you have a habit of doing - is just putting your seek power straight into the market instead of playing it first thinning your deck by -1 and then playing or putting the power as needed in the market instead - it might not seem like much and most of the time has not made much difference - but sometimes it has - just like to see a good player making correct plays not only for themselves but also for anyone watching.',
 'I ha

## Time-based Data

For both tables, there are two metrics that can be further extracted as independent features:
- Year, Month, Day of the Week of Upload
- Hour of Upload
- Video Duration in Seconds
- Video Comment Time Relative to Upload in Seconds

Let's start with the first three features in the video_metrics_df. 

For Year, Month, Day of the Week Upload, Hour of Upload we'll use the published_at column which is in YYYY-MM-DD HH-MM-SS TimeZone format. Luckily this is already in 'datetime64' by Pandas. We an use the datetime library to extract the information we need.

In [30]:
import datetime
#[x.]
print('Year : ', video_metrics_df['published_at'][0].year)
print('Month : ', video_metrics_df['published_at'][0].month)
print('Weekday : ', video_metrics_df['published_at'][0].weekday())
print('Hour : ', video_metrics_df['published_at'][0].hour)

Year :  2023
Month :  11
Weekday :  6
Hour :  5


In [31]:
video_metrics_df['published_year'] = [x.year for x in video_metrics_df['published_at']]
video_metrics_df['published_month'] = [x.month for x in video_metrics_df['published_at']]
video_metrics_df['published_weekday'] = [x.weekday() for x in video_metrics_df['published_at']]
video_metrics_df['published_hour'] = [x.hour for x in video_metrics_df['published_at']]

The duration of the video can be found under the 'duration' column and follows a slightly trickier format 'PTn(n)Hn(n)Mn(n)S where not all numbers and characters are present. If the duration is 22 seconds long, the format is 'PT22S'. If the duration is 8 minutes and 37 seconds long, the format is 'PT8M37S'.

Let try to write a function that will start from the right of the 'PT' pattern, pull the numeric values to the right of each character, and use the character pattern (H/M/S) to convert the following numeric value to seconds.

In [32]:
# example of a successful regex pattern
test = 'PT8M37S'
print(re.findall(r'[0-9]{1,2}[A-Z]', test))

['8M', '37S']


In [33]:
def pt_to_seconds(pt_string):
    '''Converts string in PT(n(n)H)(n(n)M)n(n)S) format to seconds, where () enclose optional values and H/M/S denote hours/minutes/seconds'''
    # use regex to split each string by their right most character
    pt_seconds = 0
    time_split = re.findall(r'[0-9]{1,2}[A-Z]', pt_string)
    # convert each time format to seconds and append
    for split in time_split:
        num_split = int(split[0:-1])
        if 'H' in split:
            pt_seconds += num_split * 360
        elif 'M' in split:
            pt_seconds += num_split * 60
        else:
            pt_seconds += num_split
    return(pt_seconds)

Let's test the string 'PT8M37S' which should return 8*60 + 37 = 517

In [34]:
pts_example = pt_to_seconds('PT8M37S')
print(pts_example, ' seconds')

517  seconds


In [35]:
video_metrics_df['video_duration_seconds'] = video_metrics_df['duration'].apply(lambda x: pt_to_seconds(x))
print(video_metrics_df['video_duration_seconds'])

0       83
1       22
2      123
3      517
4       89
      ... 
503     31
504     82
505     95
506    276
507     51
Name: video_duration_seconds, Length: 508, dtype: int64


Finally let's add the feature 'comment_relative_to_upload_seconds'. This will be a simple subtraction of the video_comment_time column in the video_comments_df with the published_at column in the video_metrics_df.

Because both columns have entries already in datetime format, we can use the datetime library to perform this operation.

Similar to above, let's write a function that uses the video_id from each comment to pull the published_at value from the video_metrics_df.

In [36]:
def comment_from_upload_seconds(row):
    '''extracts the video_id and datetime values from the row and finds thier respective values in the comment_metrics_df dataframe. Subtracts from both datetime values to get the time from upload'''
    c_video_id = row['video_id']
    c_datetime = row['video_comment_time']
    v_datetime = video_metrics_df[video_metrics_df['video_id'] == c_video_id]['published_at']
    c_from_v = c_datetime - v_datetime
    c_from_v_seconds = c_from_v.iloc[0].seconds
    return c_from_v_seconds

Let's validate this using the comment_id 'UgzWhoTNYW-C1nD1Lm94AaABAg' and 'Ugx5R_BBSeGsBuC3AN54AaABAg'.

In [37]:
row_1 = video_comments_df[video_comments_df['video_comment_parent_id'] == 'UgzWhoTNYW-C1nD1Lm94AaABAg']
row_2 = video_comments_df[video_comments_df['video_comment_parent_id'] == 'Ugx5R_BBSeGsBuC3AN54AaABAg']

print('Comment 1:', row_1['video_comment_time'].iloc[0])
print('Comment 2:', row_2['video_comment_time'].iloc[0])

row_1_video = video_metrics_df[video_metrics_df['video_id'] == row_1['video_id'].iloc[0]]
row_2_video = video_metrics_df[video_metrics_df['video_id'] == row_2['video_id'].iloc[0]]

print('Comment 1 video_id:', row_1_video['video_id'].iloc[0], 'and datetime', row_1_video['published_at'].iloc[0])
print('Comment 2 video_id:', row_2_video['video_id'].iloc[0], 'and datetime', row_2_video['published_at'].iloc[0])

row_1_sub = (row_1['video_comment_time'].iloc[0] - row_1_video['published_at'].iloc[0]).seconds
row_2_sub = (row_2['video_comment_time'].iloc[0] - row_2_video['published_at'].iloc[0]).seconds

print('Comment 1 expected output', row_1_sub, 'seconds')
print('Comment 2 expected output', row_2_sub, 'seconds')

Comment 1: 2023-11-15 05:10:21+00:00
Comment 2: 2023-11-14 03:48:22+00:00
Comment 1 video_id: HqNMNXa7REY and datetime 2023-11-15 03:27:18+00:00
Comment 2 video_id: P1a3DEpvFeE and datetime 2023-11-14 01:57:06+00:00
Comment 1 expected output 6183 seconds
Comment 2 expected output 6676 seconds


In [38]:
video_comments_df[video_comments_df['video_comment_parent_id'].isin(['UgzWhoTNYW-C1nD1Lm94AaABAg', 'Ugx5R_BBSeGsBuC3AN54AaABAg'])].apply(lambda x:comment_from_upload_seconds(x), axis = 1)

0    6183
1    6676
dtype: int64

We get the expected output, so let's apply that to all rows and append it to the video_comments_df:

In [40]:
video_comments_df['comments_relative_to_upload_seconds'] = video_comments_df.apply(lambda x: comment_from_upload_seconds(x), axis = 1)

## Sentiment Analysis

Let's finally perform some simple sentiment analysis on the video titles as well as comments.

Here we'll use the Natural Language Toolkit (NLTK) library which requires the following pre-processing steps prior to modelling:
1. Tokenization
2. Lowercasing
4. Noise removal
5. Spelling correction
6. Lemmatization
7. Part-of-speech tagging
8. Named entities recognition
9. Text normalization

In [60]:
import nltk
nltk_comments = video_comments_df[['video_comment']]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\justi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Before we write a function which will preprocess the text, let's validate each step.

### Tokenization and Lowercasing

First let's split the text into individual words, also known as tokens:

In [54]:
tokens_list = [nltk.word_tokenize(x.lower()) for x in nltk_comments['video_comment']]
print(tokens_list[0])

['good', 'luck', 'trying', 'out', 'the', 'charge', 'blade', '.', 'transforming', 'weapon', 'are', 'too', 'complicated', 'for', 'my', 'lance', 'brain', '.']


### Noise removal

In [63]:
import string

# Remove punctuation, stop words, whitespace, urls, html code
stop_words = nltk.corpus.stopwords.words('english')
url_pattern = r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
html_pattern = r"<[^>]+>"

filtered_token_list = []
for tokens in tokens_list:
    filtered_tokens = [x for x in tokens if x not in string.punctuation and x not in stop_words]
    filtered_tokens = [" ".join(x.strip().split()) for x in filtered_tokens]
    filtered_tokens = [re.sub(html_pattern, '', re.sub(url_pattern, '', x)) for x in filtered_tokens]
    filtered_token_list.append(filtered_tokens)

print(filtered_token_list[0])

['good', 'luck', 'trying', 'charge', 'blade', 'transforming', 'weapon', 'complicated', 'lance', 'brain']
